In [4]:
#from scratch_function.Tokenizer import SimpleTokenizer 
'''Instead to this we can use tiktoken library'''

'Instead to this we can use tiktoken library'

In [11]:
text = '''yes, you can absolutely use library to automatically extract vocabulary from text data. below are the most commonly used library and how to use them to get the vocabulary, depending on your use case:'''